In [1]:
#! /bin/python3

In [2]:
# import packages
!pip install faker
!pip install names
import json
import pandas
import ipywidgets as widgets
from IPython.display import display

In [3]:
# import os_core
from os_core.users import users
from os_core.specimen import specimen
from os_core.collection_protocoll import collection_protocol
from os_core.csv_bulk import csv_bulk
from os_core.csv_export import CSV_exporter


In [4]:
def csv_download_link(df, csv_file_name, delete_prompt=False):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    import os
    from IPython.core.display import display, HTML
    link = "http://localhost:8888/edit/work/" + csv_file_name
    df.to_csv(csv_file_name, index=False)
    display(HTML("<a href={} target=_blank>Download/Edit</a>".format(link)))
   
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

In [5]:
# import os_util
from os_util.bulk_operations import bulk_operations
from os_util.export_operations import Export_OP

In [6]:
# The URL of the Openspecimen
print('Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter')
#base_url='http://localhost:9000' # input()
base_url='http://biobank.silicolab.bibbox.org'
base_url2= 'http://biobank-7-2.silicolab.bibbox.org' 
base_url+='/openspecimen/rest/ng'
base_url2+='/openspecimen/rest/ng'
print('The whole URL for API Calls is: ' + base_url)

Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter
The whole URL for API Calls is: http://biobank.silicolab.bibbox.org/openspecimen/rest/ng


In [7]:
# Loginname and Pasword of the API-User, recommended is an extra API User
print('Loginname of the user:')
loginname='admin'#input()
print('Password of the user:')
password='Login@123'#input()

Loginname of the user:
Password of the user:


The next step is to choose the entity which will be created or updated. The following schemas are in Openspecimen available: (Name, openspecimen entity). One have to write the openspecimen entity.
('Collection Protocol', 'cp')
('Collection Protocol Registration', 'cpr')
('User', 'user')
('User Roles', 'userRoles')
('Site', 'site')
('Shipments', 'shipment')
('Institute', 'institute')
('Distribution Protocol Requirement', 'dpRequirement')
('Distribution Protocol', 'distributionProtocol')
('Distribution Order', 'distributionOrder')
('Containers', 'storageContainer')
('Container Type', 'storagecontainerType')
('Container Shipment', 'containerShipment')
('Collection Protocol Event', 'cpe')
('Master Specimen', 'masterSpecimen')
('Participant', 'participant')
('Specimen Requirement', 'sr')
('Visit', 'visit')
('Specimen Aliquot', 'specimenAliquot')
('Specimen Derivate', 'specimenDerivative')
('Specimen Disposal', 'specimenDisposal')
('Consent', 'consent')

In [8]:
# Doesn't work
schema=widgets.Dropdown(
    options= [(), ('Collection Protocol', 'cp'), ('Specimen', 'specimen'), ('Collection Protocol Registration', 'cpr'),
              ('User', 'user'), ('User Roles', 'userRoles'), ('Site', 'site'), ('Shipments', 'shipment'), 
              ('Institute', 'institute'), ('Distribution Protocol Requirement', 'dpRequirement'), 
              ('Distribution Protocol', 'distributionProtocol'), ('Distribution Order', 'distributionOrder'), 
              ('Containers', 'storageContainer'), ('Container Type', 'storagecontainerType'),
              ('Container Shipment', 'containerShipment'), ('Collection Protocol Event', 'cpe'),
              ('Master Specimen', 'masterSpecimen'), ('Participant', 'participant'), ('Specimen Requirement', 'sr'),
              ('Visit', 'visit'), ('Specimen Aliquot', 'specimenAliquot'), 
              ('Specimen Derivate', 'specimenDerivative'),('Specimen Disposal', 'specimenDisposal'),
              ('Consent', 'consent')],
    value=('Specimen', 'specimen'),
    description='Entity:')

operation = widgets.Dropdown(
    options = ["CREATE",  'UPDATE'],
              value="CREATE",
              description='Operation:')

print('Choose your Entity:')
display(schema)
print('Choose Operation:')
display(operation)

Choose your Entity:


Dropdown(description='Entity:', index=2, options=((), ('Collection Protocol', 'cp'), ('Specimen', 'specimen'),…

Choose Operation:


Dropdown(description='Operation:', options=('CREATE', 'UPDATE'), value='CREATE')

In [9]:
print("Selected Schema:")
print(schema.value)
sel_schema = schema.value[1]
print(sel_schema)
print("Selected Operation:")
sel_op = operation.value
print(sel_op)

Selected Schema:
('Collection Protocol Registration', 'cpr')
cpr
Selected Operation:
CREATE


In [10]:
#initialize csv_bulk, bulk_operations and CSV_exporter
csv_files=csv_bulk(base_url2,(loginname, password))
bulk_op=bulk_operations(base_url2, (loginname, password))
csv_export = Export_OP(base_url, (loginname, password))

In [12]:
# get necessary cp information

cp_tools_source = collection_protocol(base_url, (loginname, password))
cp_tools_target = collection_protocol(base_url2, (loginname, password))
# get source cp info
cps_source = cp_tools_source.get_all_collection_protocols()
cps_dict_source = {}
for i, item in enumerate(cps_source):
    print(item["title"])
    print(item["id"])
    cps_dict_source[i] = [item["title"], item["id"]]

# get target cp info
cps_target = cp_tools_target.get_all_collection_protocols()
cps_dict_target = {}
for i, item in enumerate(cps_target):
    print(item["title"])
    print(item["id"])
    cps_dict_target[i] = [item["title"], item["id"]]

# choose source 
print(cps_dict_source[1])
cp_id_source = cps_dict_source[1][1]
cp_ssource = cp_tools_source.get_cp_def(cp_id_source)

# choose target
print(cps_dict_target[1])
cp_id_target = cps_dict_target[1][1]
cp_target = cp_tools_target.get_collection_protocol(cp_id_target)

Main collection of Clinical-biological Biobank at INAB, comprising of all the samples.
9
Hellenic Biobank of Overweight and Obesity in Childhood and Adolescence
10
Hellenic Biobank for Parkinson's Disease
7
Main collection of Clinical-biological Biobank at INAB, comprising of all the samples.
16
Hellenic Biobank of Overweight and Obesity in Childhood and Adolescence
17
Hellenic Biobank for Parkinson's Disease
18
['Hellenic Biobank of Overweight and Obesity in Childhood and Adolescence', 10]
['Hellenic Biobank of Overweight and Obesity in Childhood and Adolescence', 17]


In [13]:
#Download Template file, Export File for data aggregation
template = csv_files.get_template(schemaname=sel_schema)
filename = sel_schema +'_template.csv'
csv_download_link(template, filename)
export = csv_export.export_file(sel_schema, cp_id_source)

In [15]:
import numpy as np
age_array = []
# crude example of age calculation
for item in export["Birth Date"]:
    age_array.append(2020 - float(item.split("-")[-1]))
age_array = np.array(age_array)
print("Age high: ",np.max(age_array))
print("Age low: ", np.min(age_array))
print("Age mean: ", age_array.mean())




Age high:  99.0
Age low:  5.0
Age mean:  51.36391068195534
{'id': 17, 'shortTitle': 'Hellenic Biobank OOCA', 'title': 'Hellenic Biobank of Overweight and Obesity in Childhood and Adolescence', 'principalInvestigator': {'id': 5, 'type': 'INSTITUTE', 'firstName': 'Evangelia', 'lastName': 'Charmandari', 'loginName': 'EvaCharm', 'domain': 'openspecimen', 'emailAddress': 'echarmand@med.uoa.gr', 'instituteName': 'Biomedical Research Foundation, Academy of Athens', 'primarySite': None, 'admin': False, 'instituteAdmin': True, 'manageForms': True, 'cpCount': 0, 'creationDate': 1607904000000, 'activityStatus': 'Active'}, 'manualPpidEnabled': False, 'specimenCentric': False, 'coordinators': [], 'cpSites': [{'id': 17, 'siteName': "Hellenic Biobank for Parkinson's Disease", 'code': None}], 'consentsWaived': False, 'anticipatedParticipantsCount': 10000, 'manualVisitNameEnabled': False, 'manualSpecLabelEnabled': False, 'bulkPartRegEnabled': False, 'barcodingEnabled': False, 'closeParentSpecimens': Fa

In [15]:
#Open file from working dir and run upload
##TODO: adding directory and name of the file
print(export)
uplo=bulk_op.bulk_import(file=export.to_csv(index=False),
                         filename='upload_cpr.csv', 
                         schemaname=sel_schema, 
                         operation=sel_op)
print(uplo)

      Identifier Study Short Title                                  PPID  \
0           2089            HBOOCA  71b0ffce-7e03-46da-97b4-65bed82d369f   
1           2090            HBOOCA  ac2987b6-3f16-40a5-be33-59810750fff4   
2           2091            HBOOCA  6e57de98-244d-4c8c-ae8e-f9665d7a0297   
3           2092            HBOOCA  94c16eb4-3fc9-4f56-8dd5-9c2eb20ecea0   
4           2093            HBOOCA  347eb46b-31db-40f2-8c86-b627c7692489   
...          ...               ...                                   ...   
3309        5398            HBOOCA  9526c7bc-1f2d-4c9f-8287-fa948d7d26be   
3310        5399            HBOOCA  551f62e4-7503-488d-bef1-8ab83c4a5301   
3311        5400            HBOOCA  5eb17362-ae48-4033-a1bf-c8e8fea08a00   
3312        5401            HBOOCA  65949acb-d5f3-451e-bdc1-399df27afa6d   
3313        5402            HBOOCA  7f74103d-2eff-4f20-a7f7-c58fbb38e351   

     Registration Date  Registration Site  External Subject ID  \
0           02-05-199

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

''

In [13]:
# Update some Specimens (operation='UPDATE')
upload_=pandas.read_csv('update_specimen.csv',index_col=False)
uplo=bulk_op.bulk_import(file=upload_.to_csv(index=False),filename='upload_specimen.csv', schemaname=schema,operation='UPDATE')
print(uplo)

  Specimen Label OS_IMPORT_STATUS  OS_ERROR_MESSAGE
0       Jupyter1          SUCCESS               NaN
1       Jupyter3          SUCCESS               NaN
2       Jupyter4          SUCCESS               NaN
